In [135]:
import os
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import scipy.io as sio
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler

from utils import check_torch, pix2vid_dataset, calculate_metrics
from utils import Units, DualCustomLoss, DualLpLoss, NeuralPix2Vid

In [136]:
NR, NT = 1272, 40
NX, NY = 40, 40
units  = Units()
folder = 'simulations_40x40'
device = check_torch()

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.3.1.post300 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3090
Torch device: cuda
------------------------------------------------------------


In [137]:
tt = np.load('{}/timesteps.npz'.format(folder))
timesteps, deltaTime = tt['timesteps'], tt['deltatime']
t0steps = timesteps[:20]
print('timesteps: {} | deltaT: {}'.format(len(timesteps), np.unique(deltaTime)))

tops2d = sio.loadmat('{}/Gt.mat'.format(folder), simplify_cells=True)['Gt']['cells']['z'].reshape(NX,NY,order='F')
print('tops2d: {}'.format(tops2d.shape))

timesteps: 40 | deltaT: [  0.5 250. ]
tops2d: (40, 40)


In [138]:
(Xt, ct, y1t, y2t, all_volumes, idx), (trainloader, validloader) = pix2vid_dataset(folder='simulations_40x40',
                                                                                   batch_size=32,
                                                                                   send_to_device=True,
                                                                                   device=device)

X: (1272, 4, 40, 40) | c: (1272, 20, 5) | y1: (1272, 20, 2, 40, 40) | y2: (1272, 20, 1, 40, 40)
----------------------------------------------------------------------------------------------------
Train - X:  torch.Size([1000, 4, 40, 40])     | c:  torch.Size([1000, 20, 5])
        y1: torch.Size([1000, 20, 2, 40, 40]) | y2: torch.Size([1000, 20, 1, 40, 40])
--------------------
Valid - X:  torch.Size([136, 4, 40, 40])     | c:  torch.Size([136, 20, 5])
        y1: torch.Size([136, 20, 2, 40, 40]) | y2: torch.Size([136, 20, 1, 40, 40])
--------------------
Test  - X:  torch.Size([136, 4, 40, 40])     | c:  torch.Size([136, 20, 5])
        y1: torch.Size([136, 20, 2, 40, 40]) | y2: torch.Size([136, 20, 1, 40, 40])


In [5]:
model = NeuralPix2Vid(device=device).to(device)
nparams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('# parameters: {:,} | device: {}'.format(nparams, model.device))

# parameters: 4,261,815 | device: cuda


In [10]:
tempxt = Xt[:20]
tempct = ct[:20]
print('Xt: {} | ct: {}'.format(tempxt.shape, tempct.shape))

tempy1t = y1t[:20]
tempy2t = y2t[:20]
print('y1t: {} | y2t: {}'.format(tempy1t.shape, tempy2t.shape))

Xt: torch.Size([20, 4, 40, 40]) | ct: torch.Size([20, 20, 5])
y1t: torch.Size([20, 20, 2, 40, 40]) | y2t: torch.Size([20, 20, 1, 40, 40])


In [12]:
tempx = Xt[:20].detach().cpu().numpy()
tempc = ct[:20].detach().cpu().numpy()
print('Xt: {} | ct: {}'.format(tempx.shape, tempc.shape))

tempy1 = y1t[:20].detach().cpu().numpy()
tempy2 = y2t[:20].detach().cpu().numpy()
print('y1t: {} | y2t: {}'.format(tempy1.shape, tempy2.shape))

Xt: (20, 4, 40, 40) | ct: (20, 20, 5)
y1t: (20, 20, 2, 40, 40) | y2t: (20, 20, 1, 40, 40)


In [105]:
tempy1p, temp2yp = model(tempxt, tempct)
tempy1p, tempy2p = tempy1p.detach().cpu().numpy(), temp2yp.detach().cpu().numpy()
print('y1p: {} | y2p: {}'.format(tempy1p.shape, temp2yp.shape))

y1p: (20, 20, 2, 40, 40) | y2p: torch.Size([20, 20, 1, 40, 40])


In [197]:
calculate_metrics(tempy1, tempy2, tempy1p, tempy2p)

---------------------------------------------------------------------------------
------------------------------------ METRICS ------------------------------------
---------------------------------------------------------------------------------
R2   - pressure: -38.0806 | saturation (inj): -inf | saturation (monitor): -3731182106014536321589915418624.0000
MSE  - pressure: 0.2121 | saturation (inj): 0.2468 | saturation (monitor): 0.2052
SSIM - pressure: 0.0051 | saturation (inj): 0.0019 | saturation (monitor): 0.0074
PSNR - pressure: 6.7344 | saturation (inj): 6.0769 | saturation (monitor): 6.8782
---------------------------------------------------------------------------------


/home/misaelmorales/miniconda3/envs/rapid/lib/python3.11/site-packages/sklearn/metrics/_regression.py:886: RuntimeWarning: overflow encountered in divide
  numerator[valid_score] / denominator[valid_score]


***
# END